In [40]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession, Row

In [ ]:
spark = SparkSession.builder.appName('dataProcessing').config('spark.executor.memory', '6g').getOrCreate()

In [ ]:
business_df = spark.read.json('Data/yelp_academic_dataset_business.json')

In [ ]:
business_df = business_df.select('business_id', 'name', 'postal_code', 'latitude', 'longitude', 'stars')
business_df = business_df.withColumnRenamed('stars', 'business_stars')

In [39]:
review_df = spark.read.json('Data/yelp_academic_dataset_review.json')
review_df = review_df.select('review_id', 'user_id', 'business_id', 'stars')
review_df = review_df.withColumnRenamed('stars', 'review_stars')

ConnectionRefusedError: [Errno 61] Connection refused

In [22]:
user_df = spark.read.json('Data/yelp_academic_dataset_user.json')
user_df = user_df.select('user_id', 'name', 'fans','friends')
user_df = user_df.withColumnRenamed('name', 'user_name')

In [23]:
merged_df = review_df.join(business_df, 'business_id', 'inner')
merged_df = merged_df.join(user_df, 'user_id', 'inner')

In [24]:
merged_df.head(5)

[Row(user_id='--Kwhcbkh7jxkhVVQZo2uQ', business_id='ORL4JE6tz3rJxVqkdKfegA', review_id='bSz0fCiKRJAB0qI9lB519A', review_stars=4.0, name='Gaylord Opryland Resort & Convention Center', postal_code='37214', latitude=36.2115923083, longitude=-86.6943191727, business_stars=3.0, user_name='Larz', fans=2, friends='MTimUubecsrWlQbdvh5YFg, R4GChcjm6g4kB6FZ78EADA, VDQwLLN2nHNE5kQpyLInVw, KEY053RQxqitaK7UVlXWWg, wrMLvsLGHAjK1SDv9vnc1A, tT65Zf4Zufu2GuVYZ3r8uA, 6cqHqMS23RvgDAPrNXeiLQ, 0wnnMyzos3hUTWMLZru_Og, k_KUouz0T_baBSxXwzrPoQ, 2WSbUNd-CRg9dAArLcOqrw, QZzKdpN7DNGDgw9Sw9MLXg, BYyIXLivleQYH9ioCn2lXg, dA3_GUsSVIeOeGrpjc4IZA, kPgNVSxXnhkyWBkqTcW45g, sEUXlW2x_YrOM-ZGX-4ZcA, UhMDnEJt1u5xqs07onDxMg, LNzPUG2ny7ouppUDs1PZFw, 4NN7vYWliPTrDlPC9X7-tA, ZZrm8nrf8nzz6qF10pxbHw, PSkfGvUwNCN6j9XDfHDlBg, cgA8WG4V795GojEi2K1AWQ, EiNtOlVYAgjbPeqg5r3Lnw, 4SMApX9CL6b_T0BBHUiWBQ, HtS4Abqtr6FSOQtOx0Wr8g, A7knuJhXtxAPmTTJqtZ73Q, w-yhXNaz-LdlEEzkLjw_9g, VxwoVH0sriy3GiwA9P9-Sg, b4pgzv34bIRNc7sSrG9t_Q, yM8jjSuh5bUAsscfY6f

In [13]:
merged_df.write.csv('Data/merged_data.csv', header=True)

In [ ]:
user_item_df = merged_df.select('user_id', 'business_id', 'review_stars')
spark.conf.set('spark.sql.pivotMaxValues', merged_df.select('business_id').distinct().count())
user_item_df = user_item_df.groupBy('user_id').pivot('business_id').sum('review_stars')
user_item_df = user_item_df.fillna(0)
user_item_df.head(5)

In [35]:
# Split the list into chunks (define your own chunk size)
from pyspark.sql.functions import col # for indicating a column using a string in the line below
chunk_size = 100  # Adjust this based on your memory constraints
distinct_business_ids = merged_df.select("business_id.id").distinct().rdd.flatMap(lambda x: x).collect() 
chunks = [distinct_business_ids[i:i + chunk_size] for i in range(0, len(distinct_business_ids), chunk_size)]
user_item_df = merged_df.select('user_id', 'business_id', 'review_stars')
# Initialize an empty DataFrame to hold the final result
final_df = None

# Iterate over each chunk and perform the pivot operation
for chunk in chunks:
    # Create a DataFrame from the chunk
    chunk_df = spark.createDataFrame([(b_id,) for b_id in chunk], ["business_id"])

    # Join with the original DataFrame
    filtered_df = user_item_df.join(chunk_df, user_item_df.business_id == chunk_df.business_id)

    # Pivot the filtered DataFrame
    pivoted_df = filtered_df.groupBy("user_id").pivot("business_id").sum("review_stars")

    # Join with the final DataFrame
    if final_df is None:
        final_df = pivoted_df
    else:
        final_df = final_df.join(pivoted_df, "user_id", "outer")


AnalysisException: [INVALID_EXTRACT_BASE_FIELD_TYPE] Can't extract a value from "business_id". Need a complex type [STRUCT, ARRAY, MAP] but got "STRING".

23/11/25 02:35:56 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1026658 ms exceeds timeout 120000 ms
23/11/25 02:35:56 WARN SparkContext: Killing executors is not supported by current scheduler.
23/11/25 02:35:56 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 